<img src="./images/antrhopic-workflows.png" width=700px>

https://www.anthropic.com/engineering/building-effective-agents

This is the example based on my codebar coaching.

We can use an agent to return back the next step as we saw in the JOKE example.

We have a list of reports and their uses and a user can ask a question with the agent returning back the most useful report to be run.

This can them be processes with more application logic.

This is ROUTING or IF/ELSE type decision making.

As we saw in 03 FAQ, we could use this to create a filter for the next agent so that it can be more selective about data retrieval.

If we combine this ROUTER as a precursor to the FAQ pattern, with one agent selecting the domain, and another agent answering the question.

This is a MULTI-AGENT system - we can have a range of design patterns that can be used to create a MULTI-AGENT system.

In [22]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from pprint import pprint
from rich.console import Console

In [23]:
console = Console()

In [24]:
# For variation use a function to get the LLM client based on the user's choice


def get_llm_client(llm_choice):

    if llm_choice == "GROQ":

        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )

        return client

    elif llm_choice == "OPENAI":

        load_dotenv()  # load environment variables from .env fil

        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        return client

    else:

        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [25]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-vIt-L1...
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [26]:
data_analysis_agents = [
    "To clean data use |DATA_CLEANING_AGENT|",
    "To run analysis on data types use |DATA_TYPES_AGENT|",
    "To do feature engineering use |FEATURE_ENGINEERING_AGENT|",
    "To detect outliers use |OUTLIER_DETECTION_AGENT|",
    "To handle missing values use |MISSING_DATA_AGENT|",
    "To normalize or standardize features use |SCALING_AGENT|",
    "To perform correlation analysis use |CORRELATION_AGENT|",
    "To create visualizations use |VISUALIZATION_AGENT|",
    "To run statistical tests use |STATISTICAL_TESTING_AGENT|",
    "To perform dimensionality reduction use |DIMENSION_REDUCTION_AGENT|",
    "To segment data into clusters use |CLUSTERING_AGENT|",
    "To build predictive models use |PREDICTIVE_MODELING_AGENT|",
    "To evaluate model performance use |MODEL_EVALUATION_AGENT|",
    "To optimize hyperparameters use |HYPERPARAMETER_TUNING_AGENT|",
    "To handle imbalanced datasets use |CLASS_IMBALANCE_AGENT|",
    "To implement time series analysis use |TIME_SERIES_AGENT|",
    "To perform text analysis use |TEXT_ANALYTICS_AGENT|",
    "To extract insights from geospatial data use |GEOSPATIAL_ANALYSIS_AGENT|",
    "To process large datasets use |BIG_DATA_PROCESSING_AGENT|",
    "To create data pipelines use |DATA_PIPELINE_AGENT|",
    "To generate synthetic data use |SYNTHETIC_DATA_AGENT|",
    "To implement cross-validation use |CROSS_VALIDATION_AGENT|",
    "To interpret complex models use |MODEL_INTERPRETATION_AGENT|",
    "To detect data drift use |DATA_DRIFT_AGENT|",
    "To run A/B tests use |AB_TESTING_AGENT|",
    "To prepare data for machine learning use |ML_PREPROCESSING_AGENT|",
    "To create ensemble models use |ENSEMBLE_MODELING_AGENT|",
    "To perform anomaly detection use |ANOMALY_DETECTION_AGENT|",
    "To optimize SQL queries use |SQL_OPTIMIZATION_AGENT|",
    "To create interactive dashboards use |DASHBOARD_CREATION_AGENT|",
    "To extract features from images use |IMAGE_FEATURE_AGENT|",
    "To implement natural language processing use |NLP_AGENT|",
    "To perform sentiment analysis use |SENTIMENT_ANALYSIS_AGENT|",
    "To do automated feature selection use |FEATURE_SELECTION_AGENT|",
    "To generate reports automatically use |AUTOMATED_REPORTING_AGENT|",
]

# Access individual tools
print(data_analysis_agents[0])  # Prints the first tool

# Count total number of data_analysis_agents
print(f"Total number of data_analysis_agents: {len(data_analysis_agents)}")
# Again, 'character and instruction' along with knowledge

To clean data use |DATA_CLEANING_AGENT|
Total number of data_analysis_agents: 35


In [27]:
# A request to the LLM is stateless so we will always need to pass all the data that is needed each time.

# `history` is just that - a record of what has gone on before so that the LLM can have context to answer the query.


# We will use GRADIO as our UI.
def chat(message, history):
    # history is part of the gradio ChatInterface and it stores previous answers
    messages = (
        [{"role": "system", "content": system_message}]
        # + history ## groq gives error as it adds metadata
        + [{"role": "user", "content": message}]
    )
    print("History is:")
    console.print(history)
    print("And messages is:")
    console.print(messages)
    # ====================
    # AI bit
    stream = client.chat.completions.create(
        model=MODEL, messages=messages, stream=True, temperature=0.0
    )

    # Just UI implementation
    response = ""
    for stream_so_far in stream:
        response += stream_so_far.choices[0].delta.content or ""
        yield response

In [28]:
system_message = """
You are a skilled data analyst and data engineer. When asked a question, you will respond with the most relevant data analysis agent from the list below.

The data analysis agents are:

"""

In [29]:
system_message += "\n" + "\n".join(data_analysis_agents)

In [30]:
system_message += """\nAlso include a reason and return output as JSON with the following structure with the report name as per context given:
{"report": " |DATA_TYPES_AGENT|", "reason": "this is a report aboit sales"}

#ONLY# JSON
"""

In [31]:
# print(system_message)

In [ ]:
# We use Gradio for a chat interface
# prompt: I am want a plane to Rome then an auto to Paris

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


History is:


[]

And messages is:


[
    {
        'role': 'system',
        'content': '\nYou are a skilled data analyst and data engineer. When asked a question, you will respond 
with the most relevant data analysis agent from the list below.\n\nThe data analysis agents are:\n\n\nTo clean data
use |DATA_CLEANING_AGENT|\nTo run analysis on data types use |DATA_TYPES_AGENT|\nTo do feature engineering use 
|FEATURE_ENGINEERING_AGENT|\nTo detect outliers use |OUTLIER_DETECTION_AGENT|\nTo handle missing values use 
|MISSING_DATA_AGENT|\nTo normalize or standardize features use |SCALING_AGENT|\nTo perform correlation analysis use
|CORRELATION_AGENT|\nTo create visualizations use |VISUALIZATION_AGENT|\nTo run statistical tests use 
|STATISTICAL_TESTING_AGENT|\nTo perform dimensionality reduction use |DIMENSION_REDUCTION_AGENT|\nTo segment data 
into clusters use |CLUSTERING_AGENT|\nTo build predictive models use |PREDICTIVE_MODELING_AGENT|\nTo evaluate model
performance use |MODEL_EVALUATION_AGENT|\nTo optimize hyperparameters use |HYPERPARAMETER_TUNING_AGENT|\nTo handle 
imbalanced datasets use |CLASS_IMBALANCE_AGENT|\nTo implement time series analysis use |TIME_SERIES_AGENT|\nTo 
perform text analysis use |TEXT_ANALYTICS_AGENT|\nTo extract insights from geospatial data use 
|GEOSPATIAL_ANALYSIS_AGENT|\nTo process large datasets use |BIG_DATA_PROCESSING_AGENT|\nTo create data pipelines 
use |DATA_PIPELINE_AGENT|\nTo generate synthetic data use |SYNTHETIC_DATA_AGENT|\nTo implement cross-validation use
|CROSS_VALIDATION_AGENT|\nTo interpret complex models use |MODEL_INTERPRETATION_AGENT|\nTo detect data drift use 
|DATA_DRIFT_AGENT|\nTo run A/B tests use |AB_TESTING_AGENT|\nTo prepare data for machine learning use 
|ML_PREPROCESSING_AGENT|\nTo create ensemble models use |ENSEMBLE_MODELING_AGENT|\nTo perform anomaly detection use
|ANOMALY_DETECTION_AGENT|\nTo optimize SQL queries use |SQL_OPTIMIZATION_AGENT|\nTo create interactive dashboards 
use |DASHBOARD_CREATION_AGENT|\nTo extract features from images use |IMAGE_FEATURE_AGENT|\nTo implement natural 
language processing use |NLP_AGENT|\nTo perform sentiment analysis use |SENTIMENT_ANALYSIS_AGENT|\nTo do automated 
feature selection use |FEATURE_SELECTION_AGENT|\nTo generate reports automatically use 
|AUTOMATED_REPORTING_AGENT|\nAlso include a reason and return output as JSON with the following structure with the 
report name as per context given:\n{"report": " |DATA_TYPES_AGENT|", "reason": "this is a report aboit 
sales"}\n\n#ONLY# JSON\n'
    },
    {'role': 'user', 'content': 'sales for last year'}
]

History is:


[
    {'role': 'user', 'metadata': None, 'content': 'sales for last year', 'options': None},
    {
        'role': 'assistant',
        'metadata': None,
        'content': '{"report": "VISUALIZATION_AGENT", "reason": "to analyze and visualize sales data for the last 
year"}',
        'options': None
    }
]

And messages is:


[
    {
        'role': 'system',
        'content': '\nYou are a skilled data analyst and data engineer. When asked a question, you will respond 
with the most relevant data analysis agent from the list below.\n\nThe data analysis agents are:\n\n\nTo clean data
use |DATA_CLEANING_AGENT|\nTo run analysis on data types use |DATA_TYPES_AGENT|\nTo do feature engineering use 
|FEATURE_ENGINEERING_AGENT|\nTo detect outliers use |OUTLIER_DETECTION_AGENT|\nTo handle missing values use 
|MISSING_DATA_AGENT|\nTo normalize or standardize features use |SCALING_AGENT|\nTo perform correlation analysis use
|CORRELATION_AGENT|\nTo create visualizations use |VISUALIZATION_AGENT|\nTo run statistical tests use 
|STATISTICAL_TESTING_AGENT|\nTo perform dimensionality reduction use |DIMENSION_REDUCTION_AGENT|\nTo segment data 
into clusters use |CLUSTERING_AGENT|\nTo build predictive models use |PREDICTIVE_MODELING_AGENT|\nTo evaluate model
performance use |MODEL_EVALUATION_AGENT|\nTo optimize hyperparameters use |HYPERPARAMETER_TUNING_AGENT|\nTo handle 
imbalanced datasets use |CLASS_IMBALANCE_AGENT|\nTo implement time series analysis use |TIME_SERIES_AGENT|\nTo 
perform text analysis use |TEXT_ANALYTICS_AGENT|\nTo extract insights from geospatial data use 
|GEOSPATIAL_ANALYSIS_AGENT|\nTo process large datasets use |BIG_DATA_PROCESSING_AGENT|\nTo create data pipelines 
use |DATA_PIPELINE_AGENT|\nTo generate synthetic data use |SYNTHETIC_DATA_AGENT|\nTo implement cross-validation use
|CROSS_VALIDATION_AGENT|\nTo interpret complex models use |MODEL_INTERPRETATION_AGENT|\nTo detect data drift use 
|DATA_DRIFT_AGENT|\nTo run A/B tests use |AB_TESTING_AGENT|\nTo prepare data for machine learning use 
|ML_PREPROCESSING_AGENT|\nTo create ensemble models use |ENSEMBLE_MODELING_AGENT|\nTo perform anomaly detection use
|ANOMALY_DETECTION_AGENT|\nTo optimize SQL queries use |SQL_OPTIMIZATION_AGENT|\nTo create interactive dashboards 
use |DASHBOARD_CREATION_AGENT|\nTo extract features from images use |IMAGE_FEATURE_AGENT|\nTo implement natural 
language processing use |NLP_AGENT|\nTo perform sentiment analysis use |SENTIMENT_ANALYSIS_AGENT|\nTo do automated 
feature selection use |FEATURE_SELECTION_AGENT|\nTo generate reports automatically use 
|AUTOMATED_REPORTING_AGENT|\nAlso include a reason and return output as JSON with the following structure with the 
report name as per context given:\n{"report": " |DATA_TYPES_AGENT|", "reason": "this is a report aboit 
sales"}\n\n#ONLY# JSON\n'
    },
    {'role': 'user', 'content': 'i want to have cross validation data '}
]